#### Simple Gen AI APP Using Langchain

In [4]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
## Langsmith Tracking
os.environ["LANGSMITH_API_KEY"]=os.getenv("LANGSMITH_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [5]:
## Data Ingestion--From the website we need to scrape the data
from langchain_community.document_loaders import WebBaseLoader

In [6]:
loader=WebBaseLoader("https://docs.smith.langchain.com/tutorials/Administrators/manage_spend")
loader

In [7]:
docs=loader.load()
docs

[Document(metadata={'source': 'https://docs.smith.langchain.com/tutorials/Administrators/manage_spend', 'title': 'LangSmith docs - Docs by LangChain', 'language': 'en'}, page_content='LangSmith docs - Docs by LangChainSkip to main contentDocs by LangChain home pageLangSmithSearch...⌘KAsk AIGitHubTry LangSmithTry LangSmithSearch...NavigationLangSmith docsGet startedObservabilityEvaluationPrompt engineeringDeploymentPlatform setupReferenceOverviewCreate an account and API keyIntegrationsPlansAccount administrationOverviewSet up hierarchyWorkload isolationManage organizations using the APIManage billingGranular usageSet up resource tagsUser managementAdditional resourcesPolly AI assistantBetaData managementAccess control & AuthenticationScalability & resilienceFAQsRegions FAQPricing FAQLangSmith statusLangSmith docsCopy pageCopy pageLangSmith is a framework-agnostic platform for developing, debugging, and deploying AI agents and LLM applications.\nIt helps you trace requests, evaluate out

In [8]:
### Load Data--> Docs-->Divide our Docuemnts into chunks dcouments-->text-->vectors-->Vector Embeddings--->Vector Store DB
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(docs)

In [9]:
documents

[Document(metadata={'source': 'https://docs.smith.langchain.com/tutorials/Administrators/manage_spend', 'title': 'LangSmith docs - Docs by LangChain', 'language': 'en'}, page_content='LangSmith docs - Docs by LangChainSkip to main contentDocs by LangChain home pageLangSmithSearch...⌘KAsk AIGitHubTry LangSmithTry LangSmithSearch...NavigationLangSmith docsGet startedObservabilityEvaluationPrompt engineeringDeploymentPlatform setupReferenceOverviewCreate an account and API keyIntegrationsPlansAccount administrationOverviewSet up hierarchyWorkload isolationManage organizations using the APIManage billingGranular usageSet up resource tagsUser managementAdditional resourcesPolly AI assistantBetaData managementAccess control & AuthenticationScalability & resilienceFAQsRegions FAQPricing FAQLangSmith statusLangSmith docsCopy pageCopy pageLangSmith is a framework-agnostic platform for developing, debugging, and deploying AI agents and LLM applications.'),
 Document(metadata={'source': 'https://

In [10]:
from langchain_openai import OpenAIEmbeddings
embeddings=OpenAIEmbeddings()

In [11]:
from langchain_community.vectorstores import FAISS
vectorstoredb=FAISS.from_documents(documents,embeddings)

In [12]:
vectorstoredb

In [13]:
## Query From a vector db
query="LangSmith has two usage limits: total traces and extended"
result=vectorstoredb.similarity_search(query)
result[0].page_content

'LangSmith docs - Docs by LangChainSkip to main contentDocs by LangChain home pageLangSmithSearch...⌘KAsk AIGitHubTry LangSmithTry LangSmithSearch...NavigationLangSmith docsGet startedObservabilityEvaluationPrompt engineeringDeploymentPlatform setupReferenceOverviewCreate an account and API keyIntegrationsPlansAccount administrationOverviewSet up hierarchyWorkload isolationManage organizations using the APIManage billingGranular usageSet up resource tagsUser managementAdditional resourcesPolly AI assistantBetaData managementAccess control & AuthenticationScalability & resilienceFAQsRegions FAQPricing FAQLangSmith statusLangSmith docsCopy pageCopy pageLangSmith is a framework-agnostic platform for developing, debugging, and deploying AI agents and LLM applications.'

In [14]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="gpt-4o")

In [19]:
## Retrieval Chain, Document chain




from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
prompt=ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context:
<context>
{context}
</context>


"""
)

document_chain=prompt|llm
document_chain

ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n\n'), additional_kwargs={})])
| ChatOpenAI(profile={'max_input_tokens': 128000, 'max_output_tokens': 16384, 'text_inputs': True, 'image_inputs': True, 'audio_inputs': False, 'video_inputs': False, 'text_outputs': True, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True, 'structured_output': True, 'image_url_inputs': True, 'pdf_inputs': True, 'pdf_tool_message': True, 'image_tool_message': True, 'tool_choice': True}, client=<openai.resources.chat.completions.completions.Completions object at 0x000001D0B6C4E9B0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 

In [20]:
from langchain_core.documents import Document
document_chain.invoke({
    "input":"LangSmith has two usage limits: total traces and extended",
    "context":[Document(page_content="LangSmith has two usage limits: total traces and extended traces. These correspond to the two metrics we've been tracking on our usage graph. ")]
})

AIMessage(content='LangSmith has two usage limits: total traces and extended traces.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 60, 'total_tokens': 73, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_a2afb0f9e7', 'id': 'chatcmpl-DAU1t73hE5oBhBNe4SJG5On3BVHlc', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c6f22-e18d-7b01-aa8b-43cb6e1b532f-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 60, 'output_tokens': 13, 'total_tokens': 73, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

However, we want the documents to first come from the retriever we just set up. That way, we can use the retriever to dynamically select the most relevant documents and pass those in for a given question.

In [21]:
### Input--->Retriever--->vectorstoredb

vectorstoredb

In [23]:
retriever=vectorstoredb.as_retriever()
retrieval_chain=retriever|document_chain


In [24]:
retrieval_chain

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001D0B6C4E4A0>, search_kwargs={})
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n\n'), additional_kwargs={})])
| ChatOpenAI(profile={'max_input_tokens': 128000, 'max_output_tokens': 16384, 'text_inputs': True, 'image_inputs': True, 'audio_inputs': False, 'video_inputs': False, 'text_outputs': True, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True, 'structured_output': True, 'image_url_inputs': True, 'pdf_inputs': True, 'pdf_tool_message': True, 'image_tool_message': True, 'tool_choice': True}, client=<openai.res

In [ ]:
## Get the response form the LLM


response = retrieval_chain.invoke(
    "LangSmith has two usage limits: total traces and extended"
)

print(response)


content='What is LangSmith according to the provided context?\n\nLangSmith is a framework-agnostic platform for developing, debugging, and deploying AI agents and LLM applications. It combines observability, evaluation, deployment, and platform setup into one integrated workflow, supporting the entire process from local development to production. LangSmith provides features like tracing requests, evaluating outputs, testing prompts, and managing deployments, and it ensures high standards of data security and privacy with HIPAA, SOC 2 Type 2, and GDPR compliance. Users can use LangSmith in a managed cloud, self-hosted environment, or hybrid setup, and it works with various frameworks and providers.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 131, 'prompt_tokens': 907, 'total_tokens': 1038, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_toke

In [27]:

response

AIMessage(content='What is LangSmith according to the provided context?\n\nLangSmith is a framework-agnostic platform for developing, debugging, and deploying AI agents and LLM applications. It combines observability, evaluation, deployment, and platform setup into one integrated workflow, supporting the entire process from local development to production. LangSmith provides features like tracing requests, evaluating outputs, testing prompts, and managing deployments, and it ensures high standards of data security and privacy with HIPAA, SOC 2 Type 2, and GDPR compliance. Users can use LangSmith in a managed cloud, self-hosted environment, or hybrid setup, and it works with various frameworks and providers.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 131, 'prompt_tokens': 907, 'total_tokens': 1038, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 

In [29]:
print(type(response))
print(response)


<class 'langchain_core.messages.ai.AIMessage'>
content='What is LangSmith according to the provided context?\n\nLangSmith is a framework-agnostic platform for developing, debugging, and deploying AI agents and LLM applications. It combines observability, evaluation, deployment, and platform setup into one integrated workflow, supporting the entire process from local development to production. LangSmith provides features like tracing requests, evaluating outputs, testing prompts, and managing deployments, and it ensures high standards of data security and privacy with HIPAA, SOC 2 Type 2, and GDPR compliance. Users can use LangSmith in a managed cloud, self-hosted environment, or hybrid setup, and it works with various frameworks and providers.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 131, 'prompt_tokens': 907, 'total_tokens': 1038, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0,